In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. magic for inline plot
# 2. magic to print version
# 3. magic so that the notebook will reload external python modules
%matplotlib inline
%load_ext watermark
%load_ext autoreload 
%autoreload 2

import time
from subprocess import call
from scipy.sparse import csr_matrix
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

# create the SparkSession class,
# which is the entry point into all functionality in Spark
spark = (SparkSession
         .builder
         .master('local[8]')
         .appName('Recommendation')
         .config(conf = SparkConf())
         .getOrCreate())

%watermark -a 'Ethen' -d -t -v -p numpy,pandas,matplotlib,scipy,pyspark

Ethen 2017-01-24 08:39:27 

CPython 3.5.2
IPython 4.2.0

numpy 1.11.3
pandas 0.18.1
matplotlib 1.5.1
scipy 0.18.1
pyspark 2.1.0+hadoop2.7


In [2]:
def read_data(file_dir):
    """
    load lastfm data and returns the dataset
    in dataframe and csr_matrix format, will
    download it if it isn't in the same folder
    """
    file_path = os.path.join(file_dir, 'usersha1-artmbid-artname-plays.tsv')
    if not os.path.isdir(file_dir):
        call(['curl', '-O', 'http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz'])
        # http://askubuntu.com/questions/25347/what-command-do-i-need-to-unzip-extract-a-tar-gz-file
        call(['tar', '-xvzf', 'lastfm-dataset-360K.tar.gz'])

    # read in triplets of user/artist/playcount from the input dataset
    col_names = ['user', 'artist', 'plays']
    data = pd.read_csv(file_path, sep = '\t', usecols = [0, 2, 3], names = col_names)

    # there are 2 rows that contains NaN value, simply drop them
    data = data.dropna(axis = 0)
    
    # map each artist and user to a unique numeric value
    data['user'] = data['user'].astype('category')
    data['artist'] = data['artist'].astype('category')

    # create a sparse matrix of using the (data, (row, col)) format
    plays = csr_matrix(( data['plays'].astype(np.float64),
                         (data['artist'].cat.codes, data['user'].cat.codes) ))
    return data, plays

In [3]:
file_dir = 'lastfm-dataset-360K'
data, plays = read_data(file_dir)
print('raw data shape:', data.shape)
print('matrix shape:', plays.shape)
data.head()

raw data shape: (17535653, 3)
matrix shape: (292364, 358868)


,user,artist,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706


In [4]:
data['user'] = data['user'].cat.codes
data['artist'] = data['artist'].cat.codes
data.head()

,user,artist,plays
0,0,45561,2137
1,0,90933,1099
2,0,185367,897
3,0,106704,717
4,0,155241,706


In [5]:
dataset = spark.createDataFrame(data, data.columns.tolist())
dataset.show(6)

+----+------+-----+
|user|artist|plays|
+----+------+-----+
|   0| 45561| 2137|
|   0| 90933| 1099|
|   0|185367|  897|
|   0|106704|  717|
|   0|155241|  706|
|   0|220129|  691|
+----+------+-----+
only showing top 6 rows



In [7]:
from pyspark.ml.recommendation import ALS
compute_start = time.time()
als = ALS(
    rank = 40,
    maxIter = 20,
    regParam = 0.01,
    implicitPrefs = True,
    alpha = 15,
    userCol = 'user', 
    itemCol = 'artist',
    ratingCol = 'plays'
).fit(dataset)
computation_time = time.time() - compute_start
computation_time / 60

Py4JJavaError: An error occurred while calling o113.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 1.0 failed 1 times, most recent failure: Lost task 7.0 in stage 1.0 (TID 8, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.lang.reflect.Array.newInstance(Array.java:75)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:1678)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1347)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:1714)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1347)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2018)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:1942)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:1808)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1353)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:201

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 49614)
----------------------------------------


Traceback (most recent call last):
  File "/Users/ethen/anaconda/lib/python3.5/socketserver.py", line 313, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/ethen/anaconda/lib/python3.5/socketserver.py", line 341, in process_request
    self.finish_request(request, client_address)
  File "/Users/ethen/anaconda/lib/python3.5/socketserver.py", line 354, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/ethen/anaconda/lib/python3.5/socketserver.py", line 681, in __init__
    self.handle()
  File "/Users/ethen/spark/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/Users/ethen/spark/python/pyspark/serializers.py", line 557, in read_int
    raise EOFError
EOFError
